<a href="https://colab.research.google.com/github/gizemnurbektas/Disaster-Tweet-Deep-Learning-Model-RNN-LSTM-/blob/main/disastertweetrnnlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split



In [ ]:
from google.colab import files
upload =files.upload()

Saving tweets.csv to tweets (1).csv


In [ ]:
df=pd.read_csv("tweets.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11370 entries, 0 to 11369
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        11370 non-null  int64 
 1   keyword   11370 non-null  object
 2   location  7952 non-null   object
 3   text      11370 non-null  object
 4   target    11370 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 444.3+ KB


,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


EDA

In [ ]:
print(df.columns)

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [ ]:
def clean_text(text):
    text = text.lower()  # küçük harf
    text = re.sub(r"http\S+", "", text)  # URL'leri kaldır
    text = re.sub(r"@\w+", "", text)  # mention'ları kaldır
    text = re.sub(r"#\w+", "", text)  # hashtag'leri kaldır
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # noktalama
    text = re.sub(r"\s+", " ", text).strip()  # fazla boşluk
    return text

df["clean_text"] = df["text"].apply(clean_text)


Büyük/küçük harf farklarını ortadan kaldırıyoruz.

Alakasız karakterleri ve sembolleri (URL, @, #, noktalama işaretleri) siliyoruz.

Böylece model için daha genel ve anlamlı bir veri oluşturuluyor.



In [ ]:
nltk.download('stopwords')
stop_words =set (stopwords.words ('english'))

def remove_stopwords (text):
  return " ".join([word for word in text.split() if word not in stop_words])

df["text"] =df["clean_text"].apply(remove_stopwords)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,keyword,location,text,target,clean_text
0,0,ablaze,NaN,communal violence bhainsa telangana stones pel...,1,communal violence in bhainsa telangana stones ...
1,1,ablaze,NaN,telangana section 144 imposed bhainsa january ...,1,telangana section 144 has been imposed in bhai...
2,2,ablaze,New York City,arsonist sets cars ablaze dealership,1,arsonist sets cars ablaze at dealership
3,3,ablaze,"Morgantown, WV",arsonist sets cars ablaze dealership,1,arsonist sets cars ablaze at dealership
4,4,ablaze,NaN,lord jesus love brings freedom pardon fill hol...,0,lord jesus your love brings freedom and pardon...


Stopwords: “the”, “is”, “a”, “and” gibi sık geçen ama çok anlam taşımayan kelimelerdir.

Bu kelimeleri kaldırarak modelin gereksiz bilgilere takılmasını önleriz.

nltk kütüphanesinden İngilizce stopwords listesi alınıp clean_text üzerinden filtreleme yapılır.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
texts = df['text'].astype(str).tolist()
labels = df['target'].values

tokenizer = Tokenizer()
tokenizer=Tokenizer(num_words=10000 ,oov_token="<OOV>");
tokenizer.fit_on_texts(df["clean_text"])
sequences = tokenizer.texts_to_sequences(df["clean_text"])

In [ ]:
sequences[:2]

[[4181,
  809,
  6,
  1953,
  2089,
  2487,
  64,
  4182,
  11,
  1234,
  760,
  7,
  74,
  760,
  7,
  1357,
  64,
  256,
  6729],
 [2089,
  2289,
  1,
  30,
  49,
  6730,
  6,
  1953,
  22,
  272,
  616,
  3,
  455,
  51,
  2488,
  2290,
  213,
  102,
  1826,
  11,
  272,
  783,
  2291]]

Metindeki her kelimeyi bir sayısal değere çeviriyoruz.

num_words=10000: En sık geçen 10.000 kelimeyle sınırlı kal (daha az veriyle daha hızlı eğitim).

oov_token: Eğitim verisinde olmayan kelimeleri <OOV> etiketiyle temsil eder.

fit_on_texts: Kelime-sayı haritasını oluşturur.

texts_to_sequences: Temizlenmiş metinleri, sayı dizilerine (sequences) dönüştürür.



In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X=pad_sequences(sequences,padding ='post' ,maxlen=100)
Y=df["target"].values

Sinir ağları sabit uzunlukta giriş ister. Bazı cümleler kısa, bazıları uzundur.

Bu yüzden tüm cümleleri eşit uzunluğa (100 kelime) getiriyoruz:

Kısa cümlelerin sonuna sıfır (0) eklenir (padding='post').

Uzun cümleler kesilir (maxlen=100).

X: Sayısal hale getirilmiş ve eşitlenmiş giriş dizileri.

y: Hedef sınıflar (etiketler), örneğin 0 = not disaster, 1 = disaster.



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced', classes=np.unique(Y_train), y=Y_train)
class_weights = dict(enumerate(weights))


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
def build_rnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=64))
    model.add(SimpleRNN(64))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
early_stop =EarlyStopping(monitor= 'val_loss', patience =3,restore_best_weights=True )

rnn_model = build_rnn_model()
rnn_history = rnn_model.fit(X_train, Y_train,
                            epochs=10,
                            batch_size=32,
                            validation_data=(X_test, Y_test),
                            class_weight=class_weights,
                            callbacks=[early_stop])


Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.4809 - loss: 0.6987 - val_accuracy: 0.6315 - val_loss: 0.6820
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.6566 - loss: 0.6850 - val_accuracy: 0.6293 - val_loss: 0.7156
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6343 - loss: 0.6848 - val_accuracy: 0.1741 - val_loss: 0.7346
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.6114 - loss: 0.6849 - val_accuracy: 0.4301 - val_loss: 0.6955


In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional, Embedding,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping



def build_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=64))
    model.add(Bidirectional(LSTM(64, dropout =0.3, recurrent_dropout =0.3)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

    early_stop =EarlyStopping(monitor= 'val_loss', patience =3,restore_best_weights=True )

lstm_model = build_lstm_model()
lstm_history = lstm_model.fit(X_train, Y_train,
                              epochs=10,
                               batch_size=32,
                              validation_data=(X_test, Y_test),
                              class_weight=class_weights,
                              callbacks=[early_stop])


Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 72s 226ms/step - accuracy: 0.6975 - loss: 0.6121 - val_accuracy: 0.8989 - val_loss: 0.2965
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 64s 225ms/step - accuracy: 0.8896 - loss: 0.3239 - val_accuracy: 0.8734 - val_loss: 0.3290
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 64s 225ms/step - accuracy: 0.9293 - loss: 0.2024 - val_accuracy: 0.8694 - val_loss: 0.3332
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 64s 222ms/step - accuracy: 0.9541 - loss: 0.1386 - val_accuracy: 0.8580 - val_loss: 0.3785


In [ ]:
def evaluate_model(model, X_test, Y_test, name="Model"):
    y_pred_probs = model.predict(X_test)
    y_pred_classes = (y_pred_probs > 0.5).astype("int32")

    print(f"\n=== {name} Sınıflandırma Raporu ===")
    print(classification_report(Y_test, y_pred_classes))

    acc = model.evaluate(X_test, Y_test)[1]
    print(f"{name} Doğruluk: %{acc * 100:.2f}")


In [ ]:
from sklearn.metrics import classification_report

evaluate_model(rnn_model, X_test, Y_test, name="RNN")
evaluate_model(lstm_model, X_test, Y_test, name="LSTM")


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

=== RNN Sınıflandırma Raporu ===
              precision    recall  f1-score   support

           0       0.84      0.68      0.75      1878
           1       0.20      0.38      0.26       396

    accuracy                           0.63      2274
   macro avg       0.52      0.53      0.51      2274
weighted avg       0.73      0.63      0.67      2274

72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6248 - loss: 0.6841
RNN Doğruluk: %63.15
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step

=== LSTM Sınıflandırma Raporu ===
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1878
           1       0.74      0.64      0.69       396

    accuracy                           0.90      2274
   macro avg       0.83      0.80      0.81      2274
weighted avg       0.89      0.90      0.90      2274

72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.8962 - loss: 0.2872
LSTM Doğruluk: %89.89


RNN Modeli Yorumu
Sınıf 0 (Not Disaster):

Precision: 0.84 → Tahminlerinin %84’ü doğru.

Recall: 0.67 → Gerçek 0 sınıflarının %67’sini yakalamış.

Sınıf 1 (Disaster):

Precision: 0.20 → 1 tahminlerinin sadece %20’si doğru.

Recall: 0.40 → Gerçek 1’lerin %40’ını yakalayabilmiş.

🔍 F1-score (1): 0.27 → Sınıf 1 için çok düşük, bu da afetleri kaçırabileceğini gösterir.

Sonuç: RNN modeli daha çok sınıf 0’a odaklanıyor, sınıf 1’i zayıf tanıyor. Accuracy yanıltıcı.

🧠 LSTM Modeli Yorumu (Geliştirilmiş)
Sınıf 0 (Not Disaster):

Precision: 0.94 → Neredeyse tüm tahminleri doğru.

Recall: 0.92 → Gerçek 0’ların %92’sini bulmuş.

Sınıf 1 (Disaster):

Precision: 0.65 → 1 tahminlerinin %65’i doğru.

Recall: 0.74 → Gerçek 1’lerin %74’ünü bulmuş.

F1-score (1): 0.69 ✅

🎯 LSTM, artık hem sınıf 0 hem sınıf 1 için güçlü tahminler yapabiliyor.

📌 Teknik Başarılar
Loss (kayıp): LSTM’de 0.3155 → RNN’e göre çok daha düşük (0.6894).

Doğruluk (accuracy):

RNN: %62

LSTM: %88.65

F1-score dengesi: LSTM her iki sınıfı da başarılı tahmin ediyor.

🧠 Ne Değişti, Neden İyi Oldu?
Muhtemelen sen şu adımları uyguladın ve bu başarı geldi:

✅ class_weight kullanıldı → Dengesiz veri telafi edildi.

✅ EarlyStopping ile overfitting önlendi.

✅ Dropout ve recurrent_dropout ile modelin ezberlemesi engellendi.

✅ (En kritik!) Bidirectional LSTM kullanımı → Cümlelerin iki yönlü bağlamı öğrenildi.

